In [52]:
from datasets import load_dataset
import pickle
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import enchant
from scipy import sparse
from math import log

In [21]:
english_dict = enchant.Dict("en_US")
stopwords = stopwords.words("english")

### Download all english wikipedia articles

In [2]:
dataset = load_dataset("wikipedia", "20220301.en", beam_runner="DirectRunner")

Found cached dataset wikipedia (C:/Users/k2002/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

### Download english stopwords

In [22]:
import nltk
nltk.download('stopwords')
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\k2002\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\k2002\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
dataset = dataset["train"]

### Keep articles containing one of those words: ["fruit", "fruits", "vegetable", "vegetables", "meat"]

In [4]:
new_dataset = []
my_words = ["fruit", "fruits", "vegetable", "vegetables", "meat"]
for site in dataset:
    tmp_text = site["text"].split(" ")
    for word in my_words:
        if word in tmp_text:
            new_dataset.append(site)
            break

In [5]:
print(len(new_dataset))

77605


In [6]:
dataset = new_dataset

### Keep articles containing one of those words: ["eat", "food"]

In [13]:
new_dataset = []
for site in dataset:
    tmp_text = site["text"].split(" ")
    if "eat" in tmp_text or "food" in tmp_text:
        new_dataset.append(site)

In [14]:
print(len(new_dataset))

21052


In [15]:
dataset = new_dataset

In [8]:
print(dataset[1454]["title"])

Tartaric acid


In [25]:
with open('dataset/my_dataset.bin', 'wb') as file:
    pickle.dump(dataset, file)

In [24]:
with open('dataset/my_dataset.bin', 'rb') as file:
    dataset = pickle.load(file)

### Reset id of every article

In [25]:
for i in range(len(dataset)):
    dataset[i]["id"] = i

In [11]:
print(dataset[1454])

{'id': 1454, 'url': 'https://en.wikipedia.org/wiki/Tartaric%20acid', 'title': 'Tartaric acid', 'text': 'Tartaric acid is a white, crystalline organic acid that occurs naturally in many fruits, most notably in grapes, but also in  bananas, tamarinds, and citrus. Its salt, potassium bitartrate, commonly known as cream of tartar, develops naturally in the process of fermentation. It is commonly mixed with sodium bicarbonate and is sold as baking powder used as a leavening agent in food preparation.  The acid itself is added to foods as an antioxidant E334 and to impart its distinctive sour taste. Naturally occurring tartaric acid is a useful raw material in organic chemical synthesis. Tartaric acid is an alpha-hydroxy-carboxylic acid, is diprotic and aldaric in acid characteristics, and is a dihydroxyl derivative of succinic acid.\n\nHistory\nTartaric acid has been known to winemakers for centuries. However, the chemical process for extraction was developed in 1769 by the Swedish chemist 

In [128]:
def simplify_string(text: str) -> list:
    def my_filter(x):
        if len(x) < 2 or any(char.isdigit() for char in x) or not english_dict.check(x):
            return False
        return True

    ps = PorterStemmer()
    text = text.lower().translate(str.maketrans("\n", " ", string.punctuation)).split(" ")
    text = [word for word in text if word not in stopwords]
    text = list(filter(my_filter, text))
    text = list(map(ps.stem, text))
    return text

In [ ]:
print(dataset[0]["text"])

In [127]:
all_words = set()
words_per_article = list()  #[id] -> dict of words
word_occurance = dict()   # word -> in_how_many_articles

In [135]:
for i in range(len(dataset)):
    text = simplify_string(dataset[i]["text"])
    word_counter = dict() # TODO better deafult_dict liblary
    for word in text:
        if word not in word_counter:
            word_counter[word] = 1
        else:
            word_counter[word] += 1
    for word in word_counter:
        all_words.add(word)
        if word not in word_occurance:
            word_occurance[word] = 1
        else:
            word_occurance[word] += 1
    words_per_article.append(word_counter)
    if i % 100 == 0:
        print(f"SAVING for i = {i}")
        with open('dataset/all_words.bin', 'wb') as file:
            pickle.dump(all_words, file)
        with open('dataset/words_per_article.bin', 'wb') as file:
            pickle.dump(words_per_article, file)
        with open('dataset/word_occurance.bin', 'wb') as file:
            pickle.dump(word_occurance, file)
    print(f"{i} / { len(dataset)}")

4114 / 21052
4115 / 21052
4116 / 21052
4117 / 21052
4118 / 21052
4119 / 21052
4120 / 21052
4121 / 21052
4122 / 21052
4123 / 21052
4124 / 21052
4125 / 21052
4126 / 21052
4127 / 21052
4128 / 21052
4129 / 21052
4130 / 21052
4131 / 21052
4132 / 21052
4133 / 21052
4134 / 21052
4135 / 21052
4136 / 21052
4137 / 21052
4138 / 21052
4139 / 21052
4140 / 21052
4141 / 21052
4142 / 21052
4143 / 21052
4144 / 21052
4145 / 21052
4146 / 21052
4147 / 21052
4148 / 21052
4149 / 21052
4150 / 21052
4151 / 21052
4152 / 21052
4153 / 21052
4154 / 21052
4155 / 21052
4156 / 21052
4157 / 21052
4158 / 21052
4159 / 21052
4160 / 21052
4161 / 21052
4162 / 21052
4163 / 21052
4164 / 21052
4165 / 21052
4166 / 21052
4167 / 21052
4168 / 21052
4169 / 21052
4170 / 21052
4171 / 21052
4172 / 21052
4173 / 21052
4174 / 21052
4175 / 21052
4176 / 21052
4177 / 21052
4178 / 21052
4179 / 21052
4180 / 21052
4181 / 21052
4182 / 21052
4183 / 21052
4184 / 21052
4185 / 21052
4186 / 21052
4187 / 21052
4188 / 21052
4189 / 21052
4190 / 21052

In [136]:
xd = False
if xd: # anty misslick
    with open('dataset/all_words.bin', 'wb') as file:
        pickle.dump(all_words, file)
    with open('dataset/words_per_article.bin', 'wb') as file:
        pickle.dump(words_per_article, file)
    with open('dataset/word_occurance.bin', 'wb') as file:
        pickle.dump(word_occurance, file)

In [26]:
with open('dataset/all_words.bin', 'rb') as file:
    all_words = pickle.load(file)
with open('dataset/words_per_article.bin', 'rb') as file:
    words_per_article = pickle.load(file)  #[id] -> dict of words
with open('dataset/word_occurance.bin', 'rb') as file:
    word_occurance = pickle.load(file)     # word -> in_how_many_articles

In [40]:
all_words = sorted(list(all_words))
all_words_indexes = dict()
for i, word in enumerate(all_words):
    all_words_indexes[word] = i

In [37]:
N = len(dataset)
M = len(all_words)
print(f"I have {N} articles and {M} words")

I have 21052 articles and 41688 words


In [48]:
words_per_article_after_idf = []
for word in words_per_article:
    words_per_article_after_idf[word] = words_per_article[word] * log(N / word_occurance[word])

TypeError: list indices must be integers or slices, not dict

In [56]:
def create_sparse_mat_and_save_to_file(filename, use_idf = False):
    row = []
    col = []
    word_counter = []
    for i in range(N):
        dict_of_words = words_per_article[i]
        for word in dict_of_words:
            col.append(i)
            row.append(all_words_indexes[word])
            if use_idf:  # TODO make this if nicer (and not repeted 1kkkkkk times)
                word_counter.append(dict_of_words[word] * log(N / word_occurance[word]))
            else:
                word_counter.append(dict_of_words[word])
        # print(f"{i} / {N}")
    my_sparse_mat = sparse.csr_matrix((word_counter, (row, col)), shape=(M, N))
    sparse.save_npz(f"dataset/{filename}", my_sparse_mat)

In [47]:
create_sparse_mat_and_save_to_file("sparse_mat_default.npz", False)

In [55]:
create_sparse_mat_and_save_to_file("sparse_mat_idf.npz", True)

In [7]:
def create_bow_for_string(text: str):
    text = simplify_string(text)
    word_counter = dict() # TODO better deafult_dict liblary
    for word in text:
        if word not in word_counter:
            word_counter[word] = 1
        else:
            word_counter[word] += 1
    # bow = [0 for _ in range(len(all_words))]
    # for word in word_counter:
    #     bow[all_words.index(word)] = word_counter[word]
    # return bow